In [59]:
import pandas as pd
import tensorflow as tf
import numpy as np
from numpy import argmax
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score

In [2]:
def wrangle(filepath):
    
    # read data into dataframe
    df = pd.read_csv(filepath)
    
    return df

In [3]:
df = wrangle("C:/Users/ufabd/Desktop/DS Lab/data/iris.csv")
print(f"data shape: {df.shape}")
df.head()

data shape: (149, 5)


,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [6]:
df.columns

Index(['5.1', '3.5', '1.4', '0.2', 'Iris-setosa'], dtype='object')

In [9]:
df["Iris-setosa"].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [10]:
X = df.drop(columns="Iris-setosa")
y = df["Iris-setosa"]

In [11]:
# Use label encoder to transform the Iris-setosa values to 0, 1, 2
y = LabelEncoder().fit_transform(y)

In [12]:
y[:3]

array([0, 0, 0])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

In [47]:
train_X, val_X, train_y, val_y = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [17]:
rfc = RandomForestClassifier(random_state=42)

In [28]:
params = {
    "n_estimators": range(2, 10)
}

In [21]:
cv =cross_val_score(rfc, X_train, y_train, cv=5, n_jobs=-1)

In [25]:
cv

array([0.92592593, 1.        , 0.96296296, 1.        , 0.96153846])

In [29]:
gs = GridSearchCV(rfc, param_grid=params, cv=5, n_jobs=-1, verbose=0)

In [30]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'n_estimators': range(2, 10)})

In [31]:
gs.best_params_

{'n_estimators': 2}

In [33]:
final_model = make_pipeline(
    StandardScaler(),
    RandomForestClassifier(n_estimators=2, random_state=42)
)

In [35]:
final_model.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=2, random_state=42))])

In [36]:
pred = final_model.predict(X_train)

In [38]:
acc = accuracy_score(y_train, pred)

In [39]:
acc

1.0

In [40]:
final_model.score(X_train, y_train)

1.0

In [41]:
test_score = final_model.score(X_test, y_test)

In [42]:
test_score

0.8666666666666667

## Using Deep MLP

In [79]:
md = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation="relu", input_shape=(X.shape[1], )),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(3, activation="softmax")
])

In [80]:
md.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [81]:
train_X.shape

(120, 4)

In [82]:
train_y.shape

(120,)

In [83]:
X.shape[1]

4

In [84]:
md.fit(train_X, train_y, epochs=150, batch_size=16, verbose=0)

In [85]:
loss, acc = md.evaluate(val_X, val_y, verbose=0)

In [86]:
print(f"Loss: {loss}, Accuracy: {acc}")

Loss: 0.062458574771881104, Accuracy: 1.0


In [87]:
row = [5.1,3.5,1.4,0.2]

In [88]:
yhat = md.predict([row])
print(f"Predicted Class: {yhat}, {argmax(yhat)}")

Predicted Class: [[9.9396557e-01 6.0082236e-03 2.6134272e-05]], 0


In [89]:
final_eva = md.evaluate(X_test, y_test)

1/1 [==============================] - 0s 31ms/step - loss: 0.1961 - accuracy: 0.9333


In [93]:
print(f"Loss: {final_eva[0]} --- Accuracy: {final_eva[1]}")

Loss: 0.19611835479736328 --- Accuracy: 0.9333333373069763
